In [31]:
import math
import csv
from scipy.spatial.transform import Rotation as R
from scipy.interpolate import interp1d
import matplotlib.pyplot as plt
import rosbag
from sensor_msgs.msg import Imu, NavSatFix
import scipy.io as sio
import numpy as np
from collections import OrderedDict
from pyproj import CRS, Transformer

In [2]:
bag = rosbag.Bag('/home/crange/dev/calib_ws/datas/calib_data_car/car.bag')
ins = []
imu = []
for topic, msg, t in bag.read_messages():
    if topic == "/novatel/oem7/inspva":
        ins.append(msg)

In [30]:
ins[0]

header: 
  seq: 24568
  stamp: 
    secs: 1681545158
    nsecs: 566694736
  frame_id: "gps"
nov_header: 
  message_name: "INSPVA"
  message_id: 508
  message_type: 0
  sequence_number: 0
  time_status: 20
  gps_week_number: 2258
  gps_week_milliseconds: 13372464
latitude: 31.271589488276586
longitude: 121.174487036288
height: 14.175300693139434
north_velocity: -0.07724594409021268
east_velocity: 0.05496245694021237
up_velocity: -0.002282923715195201
roll: 0.19605653727325859
pitch: 0.3173999522858294
azimuth: 146.32143085088416
status: 
  status: 3

In [61]:
enu_crs = CRS.from_string("+proj=utm +zone=50 +ellps=GRS80 +units=m +no_defs")
wgs84_crs = CRS.from_string("+proj=longlat +datum=WGS84 +no_defs")
trans_wgs84_to_enu = Transformer.from_crs(wgs84_crs, enu_crs)

output = []
base_enu = np.mat([trans_wgs84_to_enu.transform(ins[0].longitude, ins[0].latitude, ins[0].height)])
for i in range(len(ins)):
    timestamp = ins[i].header.stamp.secs + ins[i].header.stamp.nsecs * 1e-9
    enu = np.mat([trans_wgs84_to_enu.transform(ins[i].longitude, ins[i].latitude, ins[i].height)]) - base_enu
    euler = [ins[i].roll, ins[i].pitch, ins[i].azimuth]
    q = R.from_euler('xyz', euler, degrees=True).as_quat()
    output.append([round(timestamp, 9), 
                   round(enu[0, 0], 9), round(enu[0, 1], 9), round(enu[0, 2], 9),
                   round(q[0], 9), round(q[1], 9), round(q[2], 9), round(q[3], 9)])

In [66]:
output[0]
save_txt = 'output/car_odom.txt'
with open(save_txt, 'w', encoding='utf-8') as f:
    for i in range(len(output)):
        print("{:.{}f}\t{:.{}f}\t{:.{}f}\t{:.{}f}\t{:.{}f}\t{:.{}f}\t{:.{}f}\t{:.{}f}"
              .format(output[i][0], 9,
                      output[i][1], 9,
                      output[i][2], 9,
                      output[i][3], 9,
                      output[i][4], 9,
                      output[i][5], 9,
                      output[i][6], 9,
                      output[i][7], 9), file=f)